### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  # (what's wrong with identation?)
  # Check if the lightcurve contains enough data points after sigma clipping
  if len(mag) < 100:
      print(f"\n ################################################## Too many lightcurve points removed by sigma clipping\n")
      return
    

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.write("color_string =\n")
  output_file.write("visual_inspection_comments =\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
#folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"
#folder_path = "/home/kirill/variablestars/variable_star_classification/downloaded_lightcurves/"

zoe_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"
kirill_path = "/home/kirill/variablestars/variable_star_classification/downloaded_lightcurves/"

if os.path.exists(zoe_path):
    folder_path = zoe_path
elif os.path.exists(kirill_path):
    folder_path = kirill_path
else:
    raise Exception("Neither of the specified directories exist.")

print(f"Using folder path: {folder_path}")

Using folder path: /home/kirill/variablestars/variable_star_classification/downloaded_lightcurves/


In [7]:
#num_iterations = 1 # Why do we need it?
empty_list = []
#for i in range(7900, 8900):
for i in range(11000, 12000):

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    # Check if the length of web_format is less than 100
    if len(web_format) < 100:
        print(f"\n ################################################## Skipping index {i} due to insufficient data (length: {len(web_format)})\n")
        continue
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)
    
    try:
        classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])
    except Exception as e:
        print(f"\n ################################################## Error in classification_model for index {i}: {str(e)}\n")
        continue
        
    print(f"\n ################################################## Everything worked fine for index {i}\n")

Pulled 1 of 1


######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.4188332162787078
cusum = 0.11283232873324088
eta = 1.3945482755395306
hl_amp_ratio = 0.21008850329233628
kurtosis = -0.1876490326412399
n_points = 1155
period = 0.4904458684604272
period_SNR = 172.91291649272029
period_log10FAP = -155.53653777308094
period_uncertainty = 9.922028900130608e-05
phase_cusum = 0.28061944446920195
phase_eta = 0.3607940673029215
phi21 = 0.5998754056778929
phi31 = 1.6229924206927506
quartile31 = 0.49750984564763634
r21 = 0.4339846083480838
r31 = 0.25557489881596834
shapiro_w = 0.8451942801475525
skewness = -1.0193350205912763
slope_per10 = -0.03241212904040251
slope_per90 = 0.03416380685621322
stetson_k = 0.6776470497448969
weighted_mean = 13.700106720951833
weighted_std = 0.3474595663037501

 ################################################## Eve

/home/kirill/.conda/envs/newnewasassn/lib/python3.8/site-packages/upsilon/extract_features/extract_features.py:204: RuntimeWarning: divide by zero encountered in log10
  np.log10(pLS.getSignificance(fx, fy, nout, oversampling)[jmax])




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.33105273630336235
cusum = 0.1793976873882411
eta = 1.232933858648629
hl_amp_ratio = 0.27987932383216946
kurtosis = -0.3775406205698246
n_points = 2402
period = 0.5708794026178906
period_SNR = 276.5278525595613
period_log10FAP = -inf
period_uncertainty = 0.00011713711131189353
phase_cusum = 0.25398677951256243
phase_eta = 0.20807685296205394
phi21 = -2.522450211979903
phi31 = -1.601983071070089
quartile31 = 0.4180116978180415
r21 = 0.4555956529848813
r31 = 0.30978348950692
shapiro_w = 0.8884579539299011
skewness = -0.8727708285212984
slope_per10 = -0.019087352479908067
slope_per90 = 0.014352274611976185
stetson_k = 0.660305449593874
weighted_mean = 14.29966354903501
weighted_std = 0.29875764538942956

 ################################################## Everything worked fine for index 11

/home/kirill/.conda/envs/newnewasassn/lib/python3.8/site-packages/upsilon/extract_features/extract_features.py:204: RuntimeWarning: divide by zero encountered in log10
  np.log10(pLS.getSignificance(fx, fy, nout, oversampling)[jmax])




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.71


######### Lightcurve features #########

amplitude = 0.2136771750798029
cusum = 0.08544988237121154
eta = 2.1619711442835254
hl_amp_ratio = 0.9414084077078088
kurtosis = -1.2172253117392593
n_points = 1944
period = 0.31411609724003725
period_SNR = 256.56199521752166
period_log10FAP = -inf
period_uncertainty = 3.5470899892198204e-05
phase_cusum = 0.2347570791112103
phase_eta = 0.46282682792713215
phi21 = -1.5180134363303839
phi31 = -3.3161078202261907
quartile31 = 0.31171351699585337
r21 = 0.19868565530557822
r31 = 0.0655233846893841
shapiro_w = 0.9559565782546997
skewness = 0.05956924799559975
slope_per10 = -0.012442262891268204
slope_per90 = 0.011938925592546269
stetson_k = 0.6092681072957683
weighted_mean = 14.802120720560525
weighted_std = 0.15138332672405436

 ################################################## Everything worked fine for

...



######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.73


######### Lightcurve features #########

amplitude = 0.23931905539099665
cusum = 0.16726431947268403
eta = 1.7069305705731643
hl_amp_ratio = 1.0568462659368942
kurtosis = -1.4473381551725713
n_points = 2091
period = 0.35022519337946334
period_SNR = 289.5257569022825
period_log10FAP = -inf
period_uncertainty = 5.30992876277836e-05
phase_cusum = 0.3589201014532072
phase_eta = 0.22619169043897228
phi21 = -1.3452589467497498
phi31 = -2.5950786956812686
quartile31 = 0.3522986386500673
r21 = 0.09231622348073673
r31 = 0.07664450737817255
shapiro_w = 0.9212837815284729
skewness = 0.04532037163320853
slope_per10 = -0.022107781951786226
slope_per90 = 0.019514091121038593
stetson_k = 0.8835238921494647
weighted_mean = 13.762710936748148
weighted_std = 0.1701894242275247

 ################################################## Everything worked fine for in

/home/kirill/.conda/envs/newnewasassn/lib/python3.8/site-packages/upsilon/extract_features/extract_features.py:204: RuntimeWarning: divide by zero encountered in log10
  np.log10(pLS.getSignificance(fx, fy, nout, oversampling)[jmax])




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.78


######### Lightcurve features #########

amplitude = 0.2056080634485349
cusum = 0.12979342035491095
eta = 1.8970795542063057
hl_amp_ratio = 1.0823714540046838
kurtosis = -1.3102549294323373
n_points = 1830
period = 0.3198524020450182
period_SNR = 287.09791720236285
period_log10FAP = -inf
period_uncertainty = 4.380139791526627e-05
phase_cusum = 0.31106900044452723
phase_eta = 0.2987853199333457
phi21 = -1.4055224410463798
phi31 = -2.8722679604245487
quartile31 = 0.291366471698808
r21 = 0.09239703246778652
r31 = 0.042860876372763125
shapiro_w = 0.9433566331863403
skewness = 0.07298666195046227
slope_per10 = -0.023364140586647604
slope_per90 = 0.01933178420649439
stetson_k = 0.8585932355104491
weighted_mean = 14.318761680308668
weighted_std = 0.14711177729081856

 ################################################## Everything worked fine for in

/home/kirill/.conda/envs/newnewasassn/lib/python3.8/site-packages/upsilon/extract_features/extract_features.py:204: RuntimeWarning: divide by zero encountered in log10
  np.log10(pLS.getSignificance(fx, fy, nout, oversampling)[jmax])




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.4118694288612974
cusum = 0.24759229934443489
eta = 1.4567155472473747
hl_amp_ratio = 0.3549841506361778
kurtosis = -0.38677843641283705
n_points = 2478
period = 0.501638576350249
period_SNR = 292.97894973970807
period_log10FAP = -inf
period_uncertainty = 0.00010367289841062277
phase_cusum = 0.26467767733511927
phase_eta = 0.35221037277145323
phi21 = 0.6464666789344771
phi31 = 1.514107584593061
quartile31 = 0.5135736556282335
r21 = 0.40560229732207276
r31 = 0.29196291379212486
shapiro_w = 0.9269981384277344
skewness = -0.7446332347108522
slope_per10 = -0.00895432685381794
slope_per90 = 0.00846649895688274
stetson_k = 0.6402874818760977
weighted_mean = 15.69383095572734
weighted_std = 0.3732668538417557

 ################################################## Everything worked fine for index 